# VaR en acciones/ETF´s

## Gestión de riesgo

In [10]:
import numpy as np
import pandas as pd
import yfinance as yf


# PARAMETROS

TICKER = "TSLA"          # Ej: "AAPL", "MSFT", "QQQ", "SPY"
MONTO = 10000          # Monto de referencia (ej: 100.000)
PERIODO = "5y"          # Historial: "1y", "2y", "5y"
#


# formato argentino

def ar_num(x, dec=0):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "N/D"
    s = f"{x:,.{dec}f}"
    s = s.replace(",", "X").replace(".", ",").replace("X", ".")
    return s.split(",")[0] if dec == 0 else s

def ar_money(x, dec=2, moneda="USD"):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "N/D"
    return f"{moneda} {ar_num(x, dec)}"

def ar_pct(x, dec=1):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "N/D"
    return f"{x*100:.{dec}f}%".replace(".", ",")

def periodo_a_texto(p):
    # convierte "2y" -> "2 años", "1y" -> "1 año"
    if isinstance(p, str) and p.endswith("y"):
        n = p[:-1]
        try:
            n_int = int(n)
            return f"{n_int} año" if n_int == 1 else f"{n_int} años"
        except:
            return p
    return p


# descarga de precios

data = yf.download(
    TICKER,
    period=PERIODO,
    interval="1d",
    auto_adjust=True,
    progress=False
)

if data is None or data.empty:
    raise SystemExit(f"No pude descargar datos para {TICKER}. Probá otro ticker (ej: SPY, AAPL, QQQ).")

close = data["Close"].dropna()
rets = close.pct_change().dropna()
precio = float(close.iloc[-1])


# variaciones

def perf(days):
    if len(close) <= days:
        return np.nan
    return float(close.iloc[-1] / close.iloc[-(days + 1)] - 1)

chg_1d  = perf(1)
chg_30d = perf(21)   # ~1 mes (ruedas)
chg_1y  = perf(252)  # ~1 año (ruedas)


# rango 52 semanas

w = close.iloc[-252:] if len(close) >= 252 else close
low_52w  = float(w.min())
high_52w = float(w.max())


# riesgo (VaR / CVaR)

var95 = float(np.quantile(rets, 0.05)) if len(rets) >= 60 else np.nan
var99 = float(np.quantile(rets, 0.01)) if len(rets) >= 60 else np.nan
cvar95 = float(rets[rets <= var95].mean()) if not np.isnan(var95) else np.nan


# volatilidad anualizada y max drawdown
# volatilidad anualizada = "qué tanto se mueve en un año típico" (aprox), tomando la variabilidad diaria y escalándola.

vol_anual = float(rets.std(ddof=1) * np.sqrt(252)) if len(rets) >= 60 else np.nan

peak = close.cummax()
mdd = float(((close / peak) - 1).min())


# traducción a plata

def perdida_plata(pct):
    if pct is None or (isinstance(pct, float) and np.isnan(pct)):
        return "N/D"
    return ar_money(MONTO * abs(pct), dec=0)


# etiquetas simples

def label_vol(v):
    if np.isnan(v): return "N/D"
    if v < 0.15: return "baja"
    if v < 0.35: return "media"
    return "alta"

def label_var(v):
    if np.isnan(v): return "N/D"
    x = abs(v)
    if x < 0.015: return "bajo"
    if x < 0.035: return "medio"
    return "alto"


# salida

periodo_texto = periodo_a_texto(PERIODO)

print(f"\n📌 {TICKER.upper()} — Resumen simplificado")

print(f"- Precio actual: {ar_money(precio)}")

print("\n- Movimiento del precio:")
print(f"  • Último día: {ar_pct(chg_1d)}")
print(f"  • Últimos 30 días: {ar_pct(chg_30d)}")
print(f"  • Último año: {ar_pct(chg_1y)}")

print(f"\n- Rango 52 semanas: {ar_money(low_52w)} a {ar_money(high_52w)}")

print("\n🧯 Riesgo de mercado (VaR histórico diario)")

print(f"- VaR 95%: {ar_pct(var95)}")
print(f"  En un día malo pero 'esperable', podrías ver una baja de aprox. {perdida_plata(var95)} por cada USD {ar_num(MONTO)} invertidos.")
print(f"- VaR 99%: {ar_pct(var99)}")
print(f"  En un escenario más pesimista, la baja podría rondar unos {perdida_plata(var99)} por cada USD {ar_num(MONTO)}.")
print(f"- CVaR 95%: {ar_pct(cvar95)}")

print("\n🎢 Volatilidad y caída máxima")

print(f"- Volatilidad anualizada: {ar_pct(vol_anual)}")
print(f"  Este activo suele moverse con una volatilidad {label_vol(vol_anual)} en un año típico.")
print(f"- Caída máxima en {periodo_texto}: {ar_pct(mdd)}")
print("  Esto muestra el peor momento del período: cuánto llegó a caer desde un máximo hasta un mínimo antes de recuperarse (si se recuperó).")

print("\n✅ Conclusión rápida")

print(f"- Riesgo diario (VaR 95%): {label_var(var95)} | Volatilidad (anual): {label_vol(vol_anual)}")
print("- Si una baja diaria como la del VaR te haría vender por ansiedad, este activo debería tener poco peso en tu cartera.")
print("- Fuente: Yahoo Finance (vía yfinance). Elaboración propia: Agustín Musanti.")
print("  Análisis con fines informativos y educativos. No constituye recomendación de inversión.\n")



📌 TSLA — Resumen simplificado
- Precio actual: USD 489,88

- Movimiento del precio:
  • Último día: 3,1%
  • Últimos 30 días: 21,2%
  • Último año: 17,2%

- Rango 52 semanas: USD 221,86 a USD 489,88

🧯 Riesgo de mercado (VaR histórico diario)
- VaR 95%: -5,9%
  En un día malo pero 'esperable', podrías ver una baja de aprox. USD 586 por cada USD 10.000 invertidos.
- VaR 99%: -9,3%
  En un escenario más pesimista, la baja podría rondar unos USD 926 por cada USD 10.000.
- CVaR 95%: -8,2%

🎢 Volatilidad y caída máxima
- Volatilidad anualizada: 60,8%
  Este activo suele moverse con una volatilidad alta en un año típico.
- Caída máxima en 5 años: -73,6%
  Esto muestra el peor momento del período: cuánto llegó a caer desde un máximo hasta un mínimo antes de recuperarse (si se recuperó).

✅ Conclusión rápida
- Riesgo diario (VaR 95%): alto | Volatilidad (anual): alta
- Si una baja diaria como la del VaR te haría vender por ansiedad, este activo debería tener poco peso en tu cartera.
- Fuente:

C:\Users\agust\AppData\Local\Temp\ipykernel_23972\3438646881.py:58: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  precio = float(close.iloc[-1])
C:\Users\agust\AppData\Local\Temp\ipykernel_23972\3438646881.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(close.iloc[-1] / close.iloc[-(days + 1)] - 1)
C:\Users\agust\AppData\Local\Temp\ipykernel_23972\3438646881.py:76: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  low_52w  = float(w.min())
C:\Users\agust\AppData\Local\Temp\ipykernel_23972\3438646881.py:77: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  high_52w = float(w.max())
C:\Use